In [28]:
# Credit goes to https://sundnes.github.io/solving_odes_in_python/
import numpy as np

class ODESolver:
    def __init__(self, f):
        # Wrap user's f in a new function that always
        # converts list/tuple to array (or let array be array)
        self.f = lambda u, t: np.asarray(f(u, t), float)

    def set_initial_condition(self, U0):
        if isinstance(U0, (float,int)):  # scalar ODE
            self.neq = 1                 # no of equations
            U0 = float(U0)
        else:                            # system of ODEs
            U0 = np.asarray(U0)
            self.neq = U0.size           # no of equations
        self.U0 = U0

    def solve(self, time_points):
        self.t = np.asarray(time_points)
        N = len(self.t)
        if self.neq == 1:  # scalar ODEs
            self.u = np.zeros(N)
        else:              # systems of ODEs
            self.u = np.zeros((N,self.neq))

        # Assume that self.t[0] corresponds to self.U0
        self.u[0] = self.U0

        # Time loop
        for n in range(N-1):
            self.n = n
            self.u[n+1] = self.advance()
        return self.u, self.t

class ForwardEuler(ODESolver):
    def advance(self):
        u, f, n, t = self.u, self.f, self.n, self.t

        dt = t[n+1] - t[n]
        unew = u[n] + dt*f(u[n], t[n])
        return unew

class ExplicitMidpoint(ODESolver):
    def advance(self):
        u, f, n, t = self.u, self.f, self.n, self.t
        dt = t[n+1] - t[n]
        dt2 = dt/2.0
        k1 = f(u[n], t[n])
        k2 = f(u[n] + dt2*k1, t[n] + dt2)
        unew = u[n] + dt*k2
        return unew

class RungeKutta4(ODESolver):
    def advance(self):
        u, f, n, t = self.u, self.f, self.n, self.t
        dt = t[n+1] - t[n]
        dt2 = dt/2.0
        k1 = f(u[n], t[n])
        k2 = f(u[n] + dt2*k1, t[n] + dt2)
        k3 = f(u[n] + dt2*k2, t[n] + dt2)
        k4 = f(u[n] + dt*k3, t[n] + dt)
        unew = u[n] + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)
        return unew


In [29]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
#plt.rcParams.update({'text.usetex': True}) # import to make plots nicer
#%config InlineBackend.figure_format = 'svg' # import to make plots nicer
#from ODE_SOLVER import RungeKutta4 # for this import to work - make sure the file ODE_SOLVER is in the same file as your jupyter notebook
from scipy.integrate import solve_ivp
from matplotlib.pyplot import figure
# Set global font size for title, x-label, and y-label
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 16

# Set global font size for x and y tick labels
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

# Set global font size for the legend
plt.rcParams['legend.fontsize'] = 16

#plt.rcParams['figure.dpi'] = 300


figure(figsize=(5, 5)) # set the size of the figure
from ipywidgets import interactive



In [64]:
from re import A
def show_sol(Km1 = 1, Km2 = 1., ACt = 1., Dt = 1, r1=1, r2 = 1, r3 = 1, k1 = 1, k2=1, Km3=1, PDEt=1, r4=1, E=1, a=0.1, b=0.1):
    def CAMP_PDEP(x, t):
        def G(u, v, J, K): # equation (4)
            numerator = 2 * u * K # numerator of the fraction
            den_1 = v - u + v * J + u * K # first part of the denominator
            den_2 = -4 * (v - u) * u * K # second part of the denominator
            radicand = den_1**2 + den_2 # expression under the square root
            denominator = den_1 + np.sqrt(radicand) # compute the entire denominator
            return numerator / denominator # compute and return the fraction
        cAMP = x[0]
        PDEp = x[1]
        K1 = Km1/ACt
        K2 = Km2/ACt
        V1 = r1*cAMP
        V2 = r2*Dt    
        ACp = ACt*G(V1,V2, K1, K2) # equation (3) eq 62
        dcAMPdT = k1*ACp -k2*PDEp*cAMP #eq 64
        dPDEpdT = r3*E*((PDEt - PDEp)/Km3 + (PDEt-PDEp)) - r4*cAMP*((PDEp)/(Km3+PDEp)) #eq 60
        return [dcAMPdT, dPDEpdT]



    x0 = [a, b] # initial conditions
    solver = RungeKutta4(CAMP_PDEP)
    solver.set_initial_condition(x0)
    time_points = np.linspace(0, 500, 1000)
    x, t = solver.solve(time_points)
    R = x[:,0]; X = x[:,1]
    plt.title("Some title")
    plt.plot(t, R, label='cAMP')
    plt.plot(t, X, label='PDEp')
    plt.xlabel('Time')
    plt.ylabel('Concenration')
    plt.legend()
    plt.show()




w = interactive(show_sol, Km1=(0.1, 10, 0.01), Km2=(0.1, 10, 0.01),ACt=(0.1, 10, 0.01), Dt=(0.01, 10, 0.01), r1=(0.1, 10, 0.01), r2=(0.1, 10, 0.01), r3=(0.1, 10, 0.01), k1=(0.1, 10, 0.01), k2=(0.1, 10, 0.01), Km3=(0.1, 10, 0.01), PDEt=(0.1, 10, 0.01), r4=(0.1, 10, 0.01), E=(0.1, 10, 0.01),  a=(0.1, 10, 0.01),  b=(0.1, 10, 0.01) ,continuous_update=True)

In [65]:
w

interactive(children=(FloatSlider(value=1.0, description='Km1', max=10.0, min=0.1, step=0.01), FloatSlider(val…

In [58]:
def show_sol(Km1 = 1, Km2 = 1., ACt = 1., Dt = 1, r1=1, r2 = 1, r3 = 1, k1 = 1, k2=1, Km3=1, PDEt=1, r4=1, E=1):
    def CAMP_PDEP(x, t):
        def G(u, v, J, K): # equation (4)
            numerator = 2 * u * K # numerator of the fraction
            den_1 = v - u + v * J + u * K # first part of the denominator
            den_2 = -4 * (v - u) * u * K # second part of the denominator
            radicand = den_1**2 + den_2 # expression under the square root
            denominator = den_1 + np.sqrt(radicand) # compute the entire denominator
            return numerator / denominator # compute and return the fraction
        cAMP = x[0]
        PDEp = x[1]
        K1 = Km1/ACt
        K2 = Km2/ACt
        V1 = r1*cAMP
        V2 = r2*Dt    
        ACp = ACt*G(V1,V2, K1, K2) # equation (3) eq 62
        dcAMPdT = k1*ACp -k2*PDEp*cAMP #eq 64
        dPDEpdT = r3*E*((PDEt - PDEp)/Km3 + (PDEt-PDEp)) - r4*cAMP*((PDEp)/(Km3+PDEp)) #eq 60
        return [dcAMPdT, dPDEpdT]



    x0 = [0.1, 0.1] # initial conditions
    solver = RungeKutta4(CAMP_PDEP)
    solver.set_initial_condition(x0)
    time_points = np.linspace(0, 100, 1000)
    x, t = solver.solve(time_points)
    R = x[:,0]; X = x[:,1]
    plt.title("Some title")
    plt.plot(R,X)
    #plt.plot(t, R, label='cAMP')
    #plt.plot(t, X, label='PDEp')
    plt.xlabel('Time')
    plt.ylabel('Concenration')
    plt.legend()
    plt.show()




w2 = interactive(show_sol, Km1=(0.001, 10, 0.001), Km2=(0.001, 10, 0.01),ACt=(0.001, 10, 0.01), Dt=(0.001, 10, 0.01), r1=(0.001, 10, 0.01), r2=(0.001, 10, 0.01), r3=(0.001, 10, 0.01), k1=(0.001, 10, 0.01), k2=(0.001, 10, 0.01), Km3=(0.001, 10, 0.01), PDEt=(0.001, 10, 0.01), r4=(0.001, 10, 0.01), E=(0.001, 10, 0.01) ,continuous_update=False)

In [59]:
w2

interactive(children=(FloatSlider(value=1.0, description='Km1', max=10.0, min=0.001, step=0.001), FloatSlider(…

In [ ]:
def show_sol(k1 = 1., k2 = 1., r3 = 1., ET = 0.5, r1=1, UT=0.1, r2=0.075, J3 = 0.3, K2 = 0.3, k4 = 0.2, J4 = 0.5):
    def CAMP_PDEP(x, t):
        def G(u, v, J, K): # equation (4)
            numerator = 2 * u * K # numerator of the fraction
            den_1 = v - u + v * J + u * K # first part of the denominator
            den_2 = -4 * (v - u) * u * K # second part of the denominator
            radicand = den_1**2 + den_2 # expression under the square root
            denominator = den_1 + np.sqrt(radicand) # compute the entire denominator
            return numerator / denominator # compute and return the fraction
        Y = x[0]
        VP = G(r3 * Y, k4, J3, J4) # equation (3)
        UP = x[1]
        dYdT = k1*VP -k2*UP*Y
        dUPdT = r1*ET*((UT-UP)/(k1+(UT-UP))-r2*Y*(UP/(K2+UP)))
        return [dYdT, dUPdT]



    x0 = [0.1, 0.8] # initial conditions
    solver = RungeKutta4(CAMP_PDEP)
    solver.set_initial_condition(x0)
    time_points = np.linspace(0, 100, 1000)
    x, t = solver.solve(time_points)
    R = x[:,0]; X = x[:,1]
    plt.title("Some title")
    plt.plot(R,X)
    plt.xlabel('Time')
    plt.ylabel('Concenration')
    plt.legend()
    plt.show()




w3 = interactive(show_sol, k1=(0.1, 10, 0.01), K1=(0.1, 10, 0.01),r3=(0.1, 10, 0.01),  k2=(0.1, 10, 0.01), r1=(0.01, 10, 0.01), k3=(0.1, 10, 0.01), k4=(0.1, 10, 0.01), r2=(0.01, 10, 0.01), J4=(0.1, 10, 0.01), J3=(0.1, 10, 0.01), ET=(0.01, 10, 0.01), UT=(0.01, 10, 0.01), K2=(0.01, 10, 0.01),continuous_update=False)

In [ ]:
w3

interactive(children=(FloatSlider(value=1.0, description='k1', max=10.0, min=0.1, step=0.01), FloatSlider(valu…

In [ ]:
def show_sol(k1 = 2.87, k2 = 3.65, r3 = 4.94, ET = 2.34, r1=2.07, UT=2.62, r2=1.52, J3 =0.30, K2 =1.90, k4 = 1.35, J4 =1.46):
    def CAMP_PDEP(x, t):
        def G(u, v, J, K): # equation (4)
            numerator = 2 * u * K # numerator of the fraction
            den_1 = v - u + v * J + u * K # first part of the denominator
            den_2 = -4 * (v - u) * u * K # second part of the denominator
            radicand = den_1**2 + den_2 # expression under the square root
            denominator = den_1 + np.sqrt(radicand) # compute the entire denominator
            return numerator / denominator # compute and return the fraction
        Y = x[0]
        VP = G(r3 * Y, k4, J3, J4) # equation (3)
        UP = x[1]
        dYdT = k1*VP -k2*UP*Y
        dUPdT = r1*ET*((UT-UP)/(k1+(UT-UP))-r2*Y*(UP/(K2+UP)))
        return [dYdT, dUPdT]



    x0 = [0.1, 0.8] # initial conditions
    solver = RungeKutta4(CAMP_PDEP)
    solver.set_initial_condition(x0)
    time_points = np.linspace(0, 100, 1000)
    x, t = solver.solve(time_points)
    R = x[:,0]; X = x[:,1]
    plt.title("Some title")
    plt.plot(t,X)
    plt.plot(t,R)
    #plt.plot(R,X)
    plt.xlabel('Time')
    plt.ylabel('Concenration')
    plt.legend()
    plt.show()




w3 = interactive(show_sol, k1=(0.1, 10, 0.01), K1=(0.1, 10, 0.01),r3=(0.1, 10, 0.01),  k2=(0.1, 10, 0.01), r1=(0.01, 10, 0.01), k3=(0.1, 10, 0.01), k4=(0.1, 10, 0.01), r2=(0.01, 10, 0.01), J4=(0.1, 10, 0.01), J3=(0.1, 10, 0.01), ET=(0.01, 10, 0.01), UT=(0.01, 10, 0.01), K2=(0.01, 10, 0.01),continuous_update=False)

In [ ]:
w3

interactive(children=(FloatSlider(value=2.87, description='k1', max=10.0, min=0.1, step=0.01), FloatSlider(val…

In [69]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Your existing CAMP_PDEP function here

r1_min = 0.1
r1_max = 1.0
r2_min = 0.1
r2_max = 1.0

r1_values = np.linspace(r1_min, r1_max, num=10)
r2_values = np.linspace(r2_min, r2_max, num=10)

def run_simulation(r1, r2):
    x0 = [0.1, 0.8]  # Initial conditions
    solver = RungeKutta4(CAMP_PDEP)
    solver.set_initial_condition(x0)
    time_points = np.linspace(0, 100, 1000)
    x, t = solver.solve(time_points)
    return t, x[:, 0], x[:, 1]  # Return time points, cAMP, and PDEp

# Define parameter ranges
r1_values = np.linspace(r1_min, r1_max, num=10)
r2_values = np.linspace(r2_min, r2_max, num=10)

# Iterate over the parameter space
for r1 in r1_values:
    for r2 in r2_values:
        # Run the simulation with the current parameter values
        t, cAMP, PDEp = run_simulation(r1, r2)

        # Plot the results
        plt.figure()
        plt.title(f"Parameter sweep for r1 = {r1:.2f}, r2 = {r2:.2f}")
        plt.plot(t, cAMP, label="cAMP")
        plt.plot(t, PDEp, label="PDEp")
        plt.xlabel("Time")
        plt.ylabel("Concentration")
        plt.legend()
        plt.show()


NameError: ignored

In [71]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

Km1 = 3.02
Km2 = 1.75
ACt = 1.53
Dt = 1.12
r3 = 0.46
k1 = 1.39
k2 = 2.42
Km3 = 0.49
PDEt = 0.83
r4 = 2.11
E = 1.61


def CAMP_PDEP(t, x):
    def G(u, v, J, K): # equation (4)
        numerator = 2 * u * K # numerator of the fraction
        den_1 = v - u + v * J + u * K # first part of the denominator
        den_2 = -4 * (v - u) * u * K # second part of the denominator
        radicand = den_1**2 + den_2 # expression under the square root
        denominator = den_1 + np.sqrt(radicand) # compute the entire denominator
        return numerator / denominator # compute and return the fraction

    cAMP = x[0]
    PDEp = x[1]
    K1 = Km1/ACt
    K2 = Km2/ACt
    V1 = r1*cAMP
    V2 = r2*Dt    
    ACp = ACt*G(V1,V2, K1, K2) # equation (3) eq 62
    dcAMPdT = k1*ACp -k2*PDEp*cAMP #eq 64
    dPDEpdT = r3*E*((PDEt - PDEp)/Km3 + (PDEt-PDEp)) - r4*cAMP*((PDEp)/(Km3+PDEp)) #eq 60
    return [dcAMPdT, dPDEpdT]

# The rest of your code...


def run_simulation(r1, r2):
    x0 = [0.1, 0.8]  # Initial conditions
    time_points = np.linspace(0, 100, 1000)
    solution = solve_ivp(CAMP_PDEP, (0, 100), x0, t_eval=time_points)

    return solution.t, solution.y[0], solution.y[1]  # Return time points, cAMP, and PDEp

# Define parameter ranges
r1_min = 0.1
r1_max = 1.0
r2_min = 0.1
r2_max = 1.0

r1_values = np.linspace(r1_min, r1_max, num=10)
r2_values = np.linspace(r2_min, r2_max, num=10)

# Iterate over the parameter space
for r1 in r1_values:
    for r2 in r2_values:
        # Run the simulation with the current parameter values
        t, cAMP, PDEp = run_simulation(r1, r2)

        # Plot the results
        plt.figure()
        plt.title(f"Parameter sweep for r1 = {r1:.2f}, r2 = {r2:.2f}")
        plt.plot(t, cAMP, label="cAMP")
        plt.plot(t, PDEp, label="PDEp")
        plt.xlabel("Time")
        plt.ylabel("Concentration")
        plt.legend()
        plt.savefig(f"param_sweep_r1_{r1:.2f}_r2_{r2:.2f}.png")
        plt.show()



<ipython-input-71-c65ba88de491>:64: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


In [73]:
from numba import njit

@njit
def CAMP_PDEP(t, x, Km1, ACt, Km2, r1, r2, Dt, k1, k2, r3, E, PDEt, Km3, r4):
    def G(u, v, J, K): # equation (4)
        numerator = 2 * u * K # numerator of the fraction
        den_1 = v - u + v * J + u * K # first part of the denominator
        den_2 = -4 * (v - u) * u * K # second part of the denominator
        radicand = den_1**2 + den_2 # expression under the square root
        denominator = den_1 + np.sqrt(radicand) # compute the entire denominator
        return numerator / denominator # compute and return the fraction

    cAMP = x[0]
    PDEp = x[1]
    K1 = Km1/ACt
    K2 = Km2/ACt
    V1 = r1*cAMP
    V2 = r2*Dt    
    ACp = ACt*G(V1,V2, K1, K2) # equation (3) eq 62
    dcAMPdT = k1*ACp -k2*PDEp*cAMP #eq 64
    dPDEpdT = r3*E*((PDEt - PDEp)/Km3 + (PDEt-PDEp)) - r4*cAMP*((PDEp)/(Km3+PDEp)) #eq 60
    return np.array([dcAMPdT, dPDEpdT])

def run_simulation(Km1, ACt, Km2, r1, r2, Dt, k1, k2, r3, E, PDEt, Km3, r4):
    x0 = [0.1, 0.8]  # Initial conditions
    time_points = np.linspace(0, 100, 1000)
    solution = solve_ivp(lambda t, x: CAMP_PDEP(t, x, Km1, ACt, Km2, r1, r2, Dt, k1, k2, r3, E, PDEt, Km3, r4), (0, 100), x0, t_eval=time_points)

    return solution.t, solution.y[0], solution.y[1]  # Return time points, cAMP, and PDEp

Km1_min = 0.1
Km1_max = 1.0
ACt_min = 0.1
ACt_max = 1.0
Km2_min = 0.1
Km2_max = 1.0
r1_min = 0.1
r1_max = 1.0
r2_min = 0.1
r2_max = 1.0
Dt_min = 0.1
Dt_max = 1.0
k1_min = 0.1
k1_max = 1.0
k2_min = 0.1
k2_max = 1.0
r3_min = 0.1
r3_max = 1.0
E_min = 0.1
E_max = 1.0
PDEt_min = 0.1
PDEt_max = 1.0
Km3_min = 0.1
Km3_max = 1.0
r4_min = 0.1
r4_max = 1.0
import itertools

def parameter_sweep():
    Km1_values = np.linspace(Km1_min, Km1_max, num=3)
    ACt_values = np.linspace(ACt_min, ACt_max, num=3)
    Km2_values = np.linspace(Km2_min, Km2_max, num=3)
    r1_values = np.linspace(r1_min, r1_max, num=3)
    r2_values = np.linspace(r2_min, r2_max, num=3)
    Dt_values = np.linspace(Dt_min, Dt_max, num=3)
    k1_values = np.linspace(k1_min, k1_max, num=3)
    k2_values = np.linspace(k2_min, k2_max, num=3)
    r3_values = np.linspace(r3_min, r3_max, num=3)
    E_values = np.linspace(E_min, E_max, num=3)
    PDEt_values = np.linspace(PDEt_min, PDEt_max, num=3)
    Km3_values = np.linspace(Km3_min, Km3_max, num=3)
    r4_values = np.linspace(r4_min, r4_max, num=3)

    param_ranges = [Km1_values, ACt_values, Km2_values, r1_values, r2_values, Dt_values,
                    k1_values, k2_values, r3_values, E_values, PDEt_values, Km3_values, r4_values]

    param_combinations = itertools.product(*param_ranges)
    for params in param_combinations:
        Km1, ACt, Km2, r1, r2, Dt, k1, k2, r3, E, PDEt, Km3, r4 = params
        t, cAMP, PDEp = run_simulation(Km1, ACt, Km2, r1, r2, Dt, k1, k2, r3, E, PDEt, Km3, r4)

        # Plot the results
        plt.figure()
        plt.title(f"Parameter sweep for Km1 = {Km1:.2f}, ACt = {ACt:.2f}, Km2 = {Km2:.2f}, r1 = {r1:.2f}, r2 = {r2:.2f}, Dt = {Dt:.2f}, k1 = {k1:.2f}, k2 = {k2:.2f}, r3 = {r3:.2f}, E = {E:.2f}, PDEt = {PDEt:.2f}, Km3 = {Km3:.2f}, r4 = {r4:.2f}")
        plt.plot(t, cAMP, label="cAMP")
        plt.plot(t, PDEp, label="PDEp")
        plt.xlabel("Time")
        plt.ylabel("Concentration")
        plt.legend()
        plt.savefig(f"param_sweep_Km1_{Km1:.2f}_ACt_{ACt:.2f}_Km2_{Km2:.2f}_r1_{r1:.2f}_r2_{r2:.2f}_Dt_{Dt:.2f}_k1_{k1:.2f}_k2_{k2:.2f}_r3_{r3:.2f}_E_{E:.2f}_PDEt_{PDEt:.2f}_Km3_{Km3:.2f}_r4_{r4:.2f}.png")
        plt.close()

# Call the parameter sweep function
parameter_sweep()



KeyboardInterrupt: ignored

In [78]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from skopt import Optimizer
from skopt.space import Real
from skopt.utils import use_named_args

# Your CAMP_PDEP and run_simulation functions remain the same

# Define the objective function you want to optimize
def objective_function(params):
    Km1, ACt, Km2, r1, r2, Dt, k1, k2, r3, E, PDEt, Km3, r4 = params
    t, cAMP, PDEp = run_simulation(Km1, ACt, Km2, r1, r2, Dt, k1, k2, r3, E, PDEt, Km3, r4)
    
    # Define an objective function based on the system behavior you want to optimize
    # Example: minimize the sum of squared differences between cAMP and PDEp
    objective_value = np.sum((cAMP - PDEp)**2)
    
    return objective_value

# Define the search space for the optimization algorithm
space = [Real(0.1, 1.0) for _ in range(13)]

# Create the optimizer object
optimizer = Optimizer(dimensions=space, n_initial_points=10, acq_func="EI", random_state=42)

# Run Bayesian optimization for a fixed number of iterations
n_iterations = 50

for _ in range(n_iterations):
    # Get the next suggested parameter combination
    next_x = optimizer.ask()

    # Calculate the objective function value for the suggested parameters
    f_val = objective_function(next_x)

    # Update the optimizer with the new result
    optimizer.tell(next_x, f_val)

# Get the best parameters found by the optimization algorithm
best_params = optimizer.Xi[np.argmin(optimizer.yi)]
print("Best parameters found:", best_params)

# You can now run your simulation with the best_params and analyze the results


Best parameters found: [0.1, 0.1, 0.1, 0.1, 1.0, 0.7293535638010643, 0.6867572273596856, 0.1, 0.1, 1.0, 0.1, 0.1, 0.1]


In [76]:
pip install scikit-optimize


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 5.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

Km1 = 3.02
Km2 = 1.75
ACt = 1.53
Dt = 1.12
#r1 = 3.35
#r2 = 0.46
r3 = 0.46
k1 = 1.39
k2 = 2.42
Km3 = 0.49
PDEt = 0.83
r3 = 2.11
E = 3.39


def CAMP_PDEP(t, x):
    def G(u, v, J, K): # equation (4)
        numerator = 2 * u * K # numerator of the fraction
        den_1 = v - u + v * J + u * K # first part of the denominator
        den_2 = -4 * (v - u) * u * K # second part of the denominator
        radicand = den_1**2 + den_2 # expression under the square root
        denominator = den_1 + np.sqrt(radicand) # compute the entire denominator
        return numerator / denominator # compute and return the fraction

    cAMP = x[0]
    PDEp = x[1]
    K1 = Km1/ACt
    K2 = Km2/ACt
    V1 = r1*cAMP
    V2 = r2*Dt    
    ACp = ACt*G(V1,V2, K1, K2) # equation (3) eq 62
    dcAMPdT = k1*ACp -k2*PDEp*cAMP #eq 64
    dPDEpdT = r3*E*((PDEt - PDEp)/Km3 + (PDEt-PDEp)) - r4*cAMP*((PDEp)/(Km3+PDEp)) #eq 60
    return [dcAMPdT, dPDEpdT]
def run_simulation(r1, r2):
    x0 = [1.76, 0.81]  # Initial conditions
    time_points = np.linspace(0, 100, 1000)
    solution = solve_ivp(CAMP_PDEP, (0, 100), x0, t_eval=time_points)

    return solution.t, solution.y[0], solution.y[1]  # Return time points, cAMP, and PDEp

# Define parameter ranges
r1_min = 3.00
r1_max = 4.00
r2_min = 0.5
r2_max = 1.50

r1_values = np.linspace(r1_min, r1_max, num=100)
r2_values = np.linspace(r2_min, r2_max, num=100)

# Iterate over the parameter space
for r1 in r1_values:
    for r2 in r2_values:
        # Run the simulation with the current parameter values
        t, cAMP, PDEp = run_simulation(r1, r2)

        # Plot the results
        plt.figure()
        plt.title(f"Parameter sweep for r1 = {r1:.2f}, r2 = {r2:.2f}")
        plt.plot(t, cAMP, label="cAMP")
        plt.plot(t, PDEp, label="PDEp")
        plt.xlabel("Time")
        plt.ylabel("Concentration")
        plt.legend()
        plt.savefig(f"new_param_sweep_r1_{r1:.2f}_r2_{r2:.2f}.png")